In [1]:
import numpy as np
import copy
from tqdm import tqdm
import pandas as pd
from numpy.core.fromnumeric import transpose
from scipy.spatial import distance, distance_matrix
from custom_parser import parse_file
np.random.seed(42)
%load_ext autoreload
%autoreload 2

In [2]:
# Load the graph problem from a .tsp.txt file
# data = pd.read_csv('https://raw.githubusercontent.com/mashazya/Datathon2022/main/data/parsed_example_input.csv', sep = ',') 
data = parse_file('../data/new_testcases/priv_testcase3.def')
driver_centr_x = int((data[data.driver_type != 0].x).mean())
driver_centr_y = int((data[data.driver_type != 0].y).mean())
coordinates = pd.DataFrame(transpose([data[data['driver_type']==0]['x'],data[data['driver_type']==0]['y']]),columns=['x','y'])
coordinates.loc[-1] = [driver_centr_x, driver_centr_y]  # adding a row
coordinates.index = coordinates.index + 1  # shifting index
coordinates.sort_index(inplace=True)
# manhattan_matrix = distance_matrix(coordinates,coordinates,1)
class Chromosome():
    
    # Random generated Chromosome
    #  m - number of traveling salesmans
    def __init__(self, number_of_cities, number_of_traveling_salesman, coordinates):
        self.n = number_of_cities
        self.m = number_of_traveling_salesman
        self.coordinates = coordinates
        c = np.array(range(1,number_of_cities))
        np.random.shuffle(c)
        self.solution = np.array_split(c, self.m)
        for i in range(len(self.solution)):
            self.solution[i] = np.insert(self.solution[i],0,0)
            self.solution[i] = np.append(self.solution[i],0)
        self.fitness()
            
    # Evaluate the Chromosome - Fitness function
    #  based on 2 features: 
    #   - overall cost (cumulated from all salesman)
    #   - worst (longest) salesman cost
    def fitness(self):
        self.cost = 0
        longest_salesman_fitness = []
        longest_salesman_length = 0
        for i in range(self.m):
            salesman = self.solution[i]
            salesman_fitness = 0
            for j in range(len(salesman) - 1):
                salesman_fitness += distance.minkowski(self.coordinates.loc[salesman[j]].values, self.coordinates.loc[salesman[j+1]].values, 1)
            self.cost = self.cost + salesman_fitness
            if len(salesman) > longest_salesman_length or (len(salesman) == longest_salesman_length and salesman_fitness > self.minmax):
                longest_salesman_length = len(salesman)
                self.minmax = salesman_fitness
        self.score = self.cost + self.minmax
    
    # Mutation operator - mutates a single Traveling Salesman
    #  by swaping 2 cities
    def mutate_local(self):
        index = np.random.randint(0,self.m)
        mutant = self.solution[index]
        i,j = np.random.randint(1,len(mutant)-1), np.random.randint(1,len(mutant)-1)
        mutant[i], mutant[j] = mutant[j], mutant[i]
        old_cost = self.cost
        self.fitness()
    
    # Mutation operator - mutates 2 Traveling Salesmans
    #  by removing a city from a salesman and asigning it to the second one
    def mutate_global(self):
        for i in range(self.m):
            if len(self.solution[i]) < 3:
                print(i, self.solution[i])
        
        
        index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while index1 == index2:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while len(self.solution[index1]) < 4:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        mutant1, mutant2 = self.solution[index1], self.solution[index2]
        i,j = np.random.randint(1,len(mutant1)-1), np.random.randint(1,len(mutant2)-1)
        self.solution[index2] = np.insert(mutant2, j, mutant1[i])
        self.solution[index1] = np.delete(mutant1, i)
        old_cost = self.cost
        self.fitness()
    
    # PMX Crossover
    def crossover(self, chromosome):
        for index in range(self.m):
            salesman1, salesman2 = self.solution[index], chromosome.solution[index]
            for i in range(1,min(len(salesman1),len(salesman2))-1):
                if salesman2[i] in salesman1:
                    salesman1[i], salesman1[salesman1.tolist().index(salesman2[i])] = salesman1[salesman1.tolist().index(salesman2[i])], salesman1[i]
        self.fitness()


class Population():
    
    def __init__(self, coordinates, population_size = 50):
        self.population = []
        self.population_size = population_size
        self.coordinates = coordinates
        for i in range(population_size):
            self.population.append(Chromosome(number_of_cities = len(self.coordinates), number_of_traveling_salesman = 16, coordinates=self.coordinates))
    
    # Genetic Algorithm
    def run_genetic_algorithm(self, number_of_iterations = 1000, mutation_probability = 0.7, crossover_probability = 0.7):
        
        # Run for a fixed number of iterations
        for it in tqdm(range(number_of_iterations)):
            
            # Tournament selection
            k = self.population_size
            j = (int)(self.population_size * 0.6)
            for _ in range(self.population_size - k):
                del self.population[-np.random.randint(0,len(self.population))]
            for _ in range(k - j):
                worst_chromosome_score = self.population[0].score
                worst_chromosome_index = 0
                for i in range(1,len(self.population)):
                    if self.population[i].score > worst_chromosome_score:
                        worst_chromosome_score = self.population[i].score
                        worst_chromosome_index = i
                del self.population[-worst_chromosome_index]
                
            for _ in range(self.population_size - len(self.population)):
                self.population.append(Chromosome(number_of_cities = len(self.coordinates), number_of_traveling_salesman = 16, coordinates=self.coordinates))
            
            # Mutate globally
            for index in range(len(self.population)):
                if np.random.random(1)[0] < mutation_probability:
                    chromosome = copy.deepcopy(self.population[index])
                    chromosome.mutate_global()
                    if chromosome.score < self.population[index].score:
                        self.population[index] = chromosome
                
            # Mutate locally
            for index in range(len(self.population)):
                if np.random.random(1)[0] < mutation_probability:
                    chromosome = copy.deepcopy(self.population[index])
                    chromosome.mutate_local()
                    if chromosome.score < self.population[index].score:
                        self.population[index] = chromosome
                
            # Crossover
            for index1 in range(len(self.population)):
                if np.random.random(1)[0] < crossover_probability:
                    index2 = np.random.randint(0,len(self.population))
                    if index1 == index2:
                        index2 = np.random.randint(0,len(self.population))
                    child1 = copy.deepcopy(self.population[index1])
                    child2 = copy.deepcopy(self.population[index2])
                    child1.crossover(self.population[index2])
                    child2.crossover(self.population[index1])
                    if child1.score < self.population[index1].score:
                        self.population[index1] = child1
                    if child2.score < self.population[index2].score:
                        self.population[index2] = child2
    
    # Print the overall cost and the minmax cost of the best chromosome 
    def get_best_result(self):
        best_chromosome = self.population[0]
        for i in range(1,self.population_size):
            if self.population[i].score < best_chromosome.score:
                best_chromosome = self.population[i]
        print("Overall cost: ", best_chromosome.cost)
        print("Minmax cost: ", best_chromosome.minmax)



#____________________________________

def best_solution(size, iter,coordinates):
    pop = Population(population_size=size, coordinates=coordinates)
    pop.run_genetic_algorithm(number_of_iterations=iter)
    pop.get_best_result()

    # Iterate through population and get the best solution
    best_chromosome = pop.population[0]
    for i in range(1,pop.population_size):
        if pop.population[i].score < best_chromosome.score:
            best_chromosome = pop.population[i]
            
    # Print best solution
    routes = []
    for i in range(best_chromosome.m):
        route = []
        route.append(0)
        for j in range(1,len(best_chromosome.solution[i])):
            route.append(best_chromosome.solution[i][j])
        routes.append(route)
    return routes

def find_paths():
    driver_centr_x = int((data[data.driver_type != 0].x).mean())
    driver_centr_y = int((data[data.driver_type != 0].y).mean())
    print("data loaded")
    routes = best_solution(10, 10, coordinates)
    driver_input = pd.DataFrame(transpose([data[data['driver_type']==1]['x'],data[data['driver_type']==1]['y']]),columns=['x','y']).values
    driver_output = pd.DataFrame(transpose([data[data['driver_type']==2]['x'],data[data['driver_type']==2]['y']]),columns=['x','y']).values
    mat_dist_in= np.zeros((16,32))
    mat_dist_out= np.zeros((16,32))
    for d in range(16):
        for r in range(16):
            mat_dist_in[d][r] = distance.minkowski(driver_input[d], coordinates.values[routes[r][1]], 1)
            mat_dist_in[d][r + 16] = distance.minkowski(driver_input[d], coordinates.values[routes[r][-2]], 1)
            mat_dist_out[d][r] = distance.minkowski(driver_output[d], coordinates.values[routes[r][1]], 1)
            mat_dist_out[d][r + 16] = distance.minkowski(driver_output[d], coordinates.values[routes[r][-2]], 1)
    
    for i in range(16):
        pin = np.argmin(mat_dist_in[i])
        if pin < 16:
            #driver inici
            routes[pin][0] = i
            mat_dist_in[:,pin] = np.Inf
            mat_dist_in[:,pin+16] = np.Inf

            #driver final
            driv = np.argmin(mat_dist_out[:,pin + 16])
            routes[pin][-1] = driv
            mat_dist_out[:,pin+16] = np.Inf
            mat_dist_out[:,pin] = np.Inf
            mat_dist_out[driv,:] = np.Inf
        else:
            #driver inici
            routes[pin - 16][-1] = i
            mat_dist_in[:,pin] = np.Inf
            mat_dist_in[:,pin-16] = np.Inf

            #driver final
            driv = np.argmin(mat_dist_out[:,pin - 16])
            routes[pin-16][0] = driv
            mat_dist_out[:,pin-16] = np.Inf
            mat_dist_out[:,pin] = np.Inf
            mat_dist_out[driv,:] = np.Inf

            routes[pin-16].reverse()
    return routes
def parse_output(net_name, routes):
    drivers = data[data['driver_type'] != 0]
    pins = data[data['driver_type'] == 0]
    with open('output3_genetics.def', 'w') as f:
        for route in routes:
            f.writelines([net_name,'\n'])
            f.writelines(['  ( ', drivers['name_pin'].tolist()[route[0]], ' conn_in )\n'])
            f.writelines(['  ( ', pins['name_pin'].tolist()[route[1]-1], ' conn_out )\n'])
            f.write(';\n')
            for i in range (1,len(route)-2):
                f.writelines([net_name,'\n'])
                f.writelines(['  ( ', pins['name_pin'].tolist()[route[i]-1], ' conn_in )\n'])
                f.writelines(['  ( ', pins['name_pin'].tolist()[route[i+1]-1], ' conn_out )\n'])
                f.write(';\n')

            f.writelines([net_name,'\n'])
            f.writelines(['  ( ', pins['name_pin'].tolist()[route[-2]-1], ' conn_in )\n'])
            f.writelines(['  ( ', drivers['name_pin'].tolist()[route[-1]+16], ' conn_out )\n'])
            f.write(';\n')
        f.close()


In [3]:
routes = find_paths()
parse_output('- PRIV TEST 3', routes)

data loaded
